In [1]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import tqdm

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from itertools import chain

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import detection_utils as utils
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

1.9.0+cu102 True


In [2]:
# Register the datasets
def get_data(dataroot, file_name):
    def ret_data():
        output_path = os.path.join(dataroot, file_name)
        print(output_path)
        assert os.path.exists(output_path)
        with open(output_path, 'r') as fd:
            data = json.load(fd)
        fd.close()
        return data
    return ret_data

In [3]:
dataroot = '/usr0/tma1/datasets/nuimages/detectron_data'
train_file = 'v1.0-train_trafficcone_detectron.json'
val_file = 'val_val_trafficcone_detectron.json'
test_file = 'val_test_trafficcone_detectron.json'

In [4]:
category_dict = {0: 'traffic_cones'}
data_train = DatasetCatalog.register("nuimages_train_trafficone", get_data(dataroot, train_file))
MetadataCatalog.get("nuimages_train_trafficone").thing_classes = category_dict
data_val = DatasetCatalog.register("nuimages_val_trafficone", get_data(dataroot, val_file))
MetadataCatalog.get("nuimages_val_trafficone").thing_classes = category_dict
data_test = DatasetCatalog.register("nuimages_test_trafficone", get_data(dataroot, test_file))
MetadataCatalog.get("nuimages_test_trafficone").thing_classes = category_dict

In [5]:
date_time = '2021-10-26-01-38-57'
gt_folder = '/usr0/tma1/datasets/nuimages/detectron_data'

In [62]:
# Parameters
show_image=False
output_dir='/usr0/tma1/datasets/nuimages/object_detection_annotations'
predicted_dir='/usr0/tma1/traffic_cone_detection/output'
visualize_dataset_name = 'nuimages_test_trafficone'
split = 'test'

scale=5.0
predicted_output_folder = os.path.join(os.path.join(predicted_dir, date_time), 'predicted_output')
if not os.path.exists(predicted_output_folder):
    os.mkdir(predicted_output_folder)
    
prediction_name = date_time + '_' + visualize_dataset_name
predicted_output_folder = os.path.join(predicted_output_folder, prediction_name)
if not os.path.exists(predicted_output_folder):
    os.mkdir(predicted_output_folder)
    
output_folder = os.path.join(os.path.join(predicted_dir, date_time), 'evaluation_output')
output_split_folder = os.path.join(output_folder, visualize_dataset_name)
if not os.path.exists(output_split_folder):
    os.mkdir(output_split_folder)

/usr0/tma1/traffic_cone_detection/output/2021-10-26-01-38-57/evaluation_output/nuimages_test_trafficone


In [7]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.OUTPUT_DIR = os.path.join('/usr0/tma1/traffic_cone_detection/output', date_time)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0076499.pth")
# cfg.MODEL.DEVICE='cpu'
predictor = DefaultPredictor(cfg)

In [66]:
def format_instance(instance, category_dict):
    class_name = category_dict[instance.get('pred_classes').data.cpu().numpy()[0]]
    pred_boxes = instance.get('pred_boxes').tensor.data.cpu().numpy()[0]
    score = instance.get('scores').data.cpu().numpy()[0]
    return '{} {} {} {} {} {}'.format(class_name, score, pred_boxes[0], pred_boxes[1], pred_boxes[2], pred_boxes[3])

In [67]:
dicts = list(chain.from_iterable([DatasetCatalog.get(k) for k in [visualize_dataset_name]]))
metadata = MetadataCatalog.get(visualize_dataset_name)
i = 0;
for dic in tqdm.tqdm(dicts):
    im = utils.read_image(dic["file_name"], "RGB")
    outputs = predictor(im)
    predictions = []
    for i in range(len(outputs['instances'])):
        predictions.append(format_instance(outputs['instances'][i], category_dict))
    
    img_name = dic["file_name"].split('/')[-1]
    txt_name = img_name.split('.')[0] + '.txt'
    txt_name = os.path.join(output_split_folder, txt_name)
    
    # Write result to folder
    with open(txt_name, 'w+') as fd:
        fd.write('\n'.join(predictions))



/usr0/tma1/datasets/nuimages/detectron_data/val_test_trafficcone_detectron.json


100%|██████████| 2263/2263 [05:55<00:00,  6.37it/s]


Create ground truth formats

In [77]:
gt_eval = '/usr0/tma1/datasets/nuimages/detectron_data/evaluation_ground_truth'
gt_eval_split = os.path.join(gt_eval, visualize_dataset_name)
if not os.path.exists(gt_eval_split):
    os.mkdir(gt_eval_split)

with open(os.path.join(dataroot, test_file), 'r') as fd:
    data = json.load(fd)
    i = 0;
    for d in tqdm.tqdm(data):
        i += 1
        img_name = d['file_name'].split('/')[-1]
        txt_name = img_name.split('.')[0] + '.txt'
        annotation = []
        for anno in d['annotations']:
            pred_boxes = anno['bbox']
            class_name = category_dict[anno['category_id']]
            rslt = '{} {} {} {} {}'.format(class_name, pred_boxes[0], pred_boxes[1], pred_boxes[2], pred_boxes[3])
            annotation.append(rslt)
        
        out_file = os.path.join(gt_eval_split, txt_name)
        with open(out_file, 'w+') as fd:
            fd.write('\n'.join(annotation))

100%|██████████| 2263/2263 [00:00<00:00, 28436.94it/s]
